# Translation  (Seq2Seq)

Style transfer: Creating a model that translates texts written in a certain style to another (e.g., formal to casual or Shakespearean English to modern English)

Generative question answering: Creating a model that generates answers to questions, given a context

In [1]:
!pip install -q git+https://github.com/huggingface/transformers
!pip install -q git+https://github.com/huggingface/accelerate
!pip install -q datasets evaluate wandb sentencepiece

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 61.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 60.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 85.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.6 MB/s eta 0:00:00
     ━━━━━

In [22]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
import wandb

wandb.init(project="t5-small-finetuned-en-to-fr-accelerate", entity="tchoud8")

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Split data into Train & Validation

In [4]:
from datasets import load_dataset

# raw_datasets = load_dataset("kde4", lang1="en", lang2="fr")
raw_datsets = load_dataset("pritam3355/en-ga-dataset")
raw_datasets["train"] = raw_datasets["train"].shuffle(seed=42).select(range(40000))
split_datasets = raw_datasets["train"].train_test_split(train_size=0.9, seed=42)
split_datasets["validation"] = split_datasets.pop("test")
split_datasets

Generating train split:   0%|          | 0/210173 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 36000
    })
    validation: Dataset({
        features: ['id', 'translation'],
        num_rows: 4000
    })
})

In [ ]:
model_checkpoint = "t5-small"

model_name = "t5-small-finetuned-en-to-fr-accelerate"

In [5]:
from transformers import T5Tokenizer, T5ForConditionalGeneration



tokenizer = T5Tokenizer.from_pretrained(model_checkpoint)
model = T5ForConditionalGeneration.from_pretrained(model_checkpoint, return_dict=True)

input = split_datasets["train"][1]["translation"]["en"]

# You can also use "translate English to French" and "translate English to Romanian"
input_ids = tokenizer("translate: English to French: "+input, return_tensors="pt").input_ids  # Batch size 1

outputs = model.generate(input_ids)

decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)



You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565, and set the legacy attribute accordingly.


In [6]:
print(split_datasets["train"][1]["translation"]["en"])
print("-"*80)
print(decoded)

Error in opening output file
--------------------------------------------------------------------------------
erreur dans l'ouverture du fichier de sortie


For multilingual tokenizer such as mBART, mBART-50, or M2M100, you will need to set the language codes of your inputs and targets in the tokenizer by setting tokenizer.src_lang and tokenizer.tgt_lang to the right values.

In [7]:
en_sentence = split_datasets["train"][1]["translation"]["en"]
fr_sentence = split_datasets["train"][1]["translation"]["fr"]

inputs = tokenizer("translate: English to French: "+en_sentence, text_target=fr_sentence)
inputs

{'input_ids': [13959, 10, 1566, 12, 2379, 10, 848, 52, 127, 16, 2101, 3911, 1042, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [848, 60, 450, 3, 26, 31, 14398, 146, 13785, 20, 13939, 1]}

As we can see, using the English tokenizer to preprocess a French sentence results in a lot more tokens, since the tokenizer doesn’t know any French words (except those that also appear in the English language, like “discussion”).

Check if the output contains the input IDs associated with the English sentence, while the IDs associated with the French one are stored in the labels field. If you forget to indicate that you are tokenizing labels, they will be tokenized by the input tokenizer,this is specially the case with pre-trained multilingual models,But here we're using T5-small so no need to worry

In [8]:
# wrong_targets = tokenizer(fr_sentence)
# print(tokenizer.convert_ids_to_tokens(wrong_targets["input_ids"]))
# print(tokenizer.convert_ids_to_tokens(inputs["labels"]))

In [9]:


max_length = 128


def preprocess_function(examples):
    inputs = ["translate: English to French: "+ex["en"] for ex in examples["translation"]]
    targets = [ex["fr"] for ex in examples["translation"]]
    model_inputs = tokenizer(
        inputs, text_target=targets, max_length=max_length, truncation=True
    )
    return model_inputs

tokenized_datasets = split_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=split_datasets["train"].column_names,
)

Map:   0%|          | 0/36000 [00:00<?, ? examples/s]

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

In [10]:
from transformers import AutoModelForSeq2SeqLM,AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors="pt")

In [11]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)



list of examples from tokenized training set

In [12]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(1, 3)])
batch.keys()

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])

In [13]:
batch["labels"]

tensor([[  848,    60,   450,     3,    26,    31, 14398,   146, 13785,    20,
         13939,     1],
        [27370,   599,     7,    61,     3,   184,  7172,  3328,   117,    10,
             1,  -100]])

In [14]:
batch["decoder_input_ids"]

tensor([[    0,   848,    60,   450,     3,    26,    31, 14398,   146, 13785,
            20, 13939],
        [    0, 27370,   599,     7,    61,     3,   184,  7172,  3328,   117,
            10,     1]])

In [15]:
for i in range(1, 3):
    print(tokenized_datasets["train"][i]["labels"])

[848, 60, 450, 3, 26, 31, 14398, 146, 13785, 20, 13939, 1]
[27370, 599, 7, 61, 3, 184, 7172, 3328, 117, 10, 1]


The traditional metric used for translation is the BLEU score, introduced in a 2002 article by Kishore Papineni et al. The BLEU score evaluates how close the translations are to their labels. It does not measure the intelligibility or grammatical correctness of the model’s generated outputs, but uses statistical rules to ensure that all the words in the generated outputs also appear in the targets. In addition, there are rules that penalize repetitions of the same words if they are not also repeated in the targets (to avoid the model outputting sentences like "the the the the the") and output sentences that are shorter than those in the targets (to avoid the model outputting sentences like "the").



In [16]:
!pip install -q sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 2.5 MB/s eta 0:00:00


One weakness with BLEU is that it expects the text to already be tokenized, which makes it difficult to compare scores between models that use different tokenizers. So instead, the most commonly used metric for benchmarking translation models today is SacreBLEU, which addresses this weakness (and others) by standardizing the tokenization step.

In [17]:
import evaluate

metric = evaluate.load("sacrebleu")

This metric will take texts as inputs and targets. It is designed to accept several acceptable targets, as there are often multiple acceptable translations of the same sentence — the dataset we’re using only provides one, but it’s not uncommon in NLP to find datasets that give several sentences as labels. So, the predictions should be a list of sentences, but the references should be a list of lists of sentences.

In [18]:
predictions = ["This plugin lets you translate web pages between several languages automatically."]
references = [["This plugin allows you to automatically translate web pages between several languages."]]
metric.compute(predictions=predictions, references=references)

{'score': 46.750469682990165,
 'counts': [11, 6, 4, 3],
 'totals': [12, 11, 10, 9],
 'precisions': [91.66666666666667,
  54.54545454545455,
  40.0,
  33.333333333333336],
 'bp': 0.9200444146293233,
 'sys_len': 12,
 'ref_len': 13}

The score can go from 0 to 100, and higher is better.

To get from the model outputs to texts the metric can use, we will use the tokenizer.batch_decode() method. We just have to clean up all the -100s in the labels (the tokenizer will automatically do the same for the padding token)

In [19]:
import numpy as np


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

We don’t set any regular evaluation, as evaluation takes a while; we will just evaluate our model once before training and after.

We set fp16=True, which speeds up training on modern GPUs.

We set predict_with_generate=True,

In [20]:
from transformers import Seq2SeqTrainingArguments

args = Seq2SeqTrainingArguments(
    model_name, # output_dir
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

# Fine Tuning using Trainer API

In [23]:
# from transformers import Seq2SeqTrainer

# trainer = Seq2SeqTrainer(
#     model,
#     args,
#     train_dataset=tokenized_datasets["train"],
#     eval_dataset=tokenized_datasets["validation"],
#     data_collator=data_collator,
#     tokenizer=tokenizer,
#     compute_metrics=compute_metrics,
# )

Before training, we’ll first look at the score our model gets, to double-check that we’re not making things worse with our fine-tuning.

In [24]:
# trainer.evaluate(max_length=max_length)

In [25]:
# trainer.train()

In [26]:
# trainer.evaluate(max_length=max_length)

In [27]:
# trainer.push_to_hub(tags="translation", commit_message="Training complete")

# Fine tuning using Accelerator

In [28]:
from torch.utils.data import DataLoader

tokenized_datasets.set_format("torch")
train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], collate_fn=data_collator, batch_size=8
)

we reinstantiate our model, to make sure we’re not continuing the fine-tuning from before but starting from the pretrained model again:

In [29]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [30]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [31]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

Now that we have sent our train_dataloader to accelerator.prepare(), we can use its length to compute the number of training steps. Remember we should always do this after preparing the dataloader, as that method will change the length of the DataLoader. We use a classic linear schedule from the learning rate to 0

In [32]:
from transformers import get_scheduler

num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [33]:
from huggingface_hub import Repository, get_full_repo_name


repo_name = get_full_repo_name(model_name)
repo_name

'pritam3355/t5-finetuned-en-to-fr-accelerate'

In [34]:
output_dir = model_name
repo = Repository(output_dir, clone_from=repo_name)

Cloning https://huggingface.co/pritam3355/t5-finetuned-en-to-fr-accelerate into local empty directory.


In [35]:
def postprocess(predictions, labels):
    predictions = predictions.cpu().numpy()
    labels = labels.cpu().numpy()

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    return decoded_preds, decoded_labels

The first thing to note is that we use the generate() method to compute predictions, but this is a method on our base model, not the wrapped model. Accelerate created in the prepare() method. That’s why we unwrap the model first, then call this method.

In [39]:
from tqdm.auto import tqdm
import torch

progress_bar = tqdm(range(num_training_steps))
batch_size = 64

for epoch in range(num_train_epochs):
    # Training
    model.train()
    total_train_loss = 0.0
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        total_train_loss += loss.item()  # Add current batch loss to total training loss
        progress_bar.update(1)

    # Calculate average training loss for the epoch
    avg_train_loss = total_train_loss / len(train_dataloader)

    # Evaluation
    model.eval()
    losses = []
    for batch in tqdm(eval_dataloader):
        with torch.no_grad():
            generated_tokens = accelerator.unwrap_model(model).generate(
                batch["input_ids"],
                attention_mask=batch["attention_mask"],
                max_length=128,
            )
            outputs = model(**batch)
        loss = outputs.loss
        losses.append(accelerator.gather(loss.repeat(batch_size)))

        labels = batch["labels"]

        # Necessary to pad predictions and labels for being gathered
        generated_tokens = accelerator.pad_across_processes(
            generated_tokens, dim=1, pad_index=tokenizer.pad_token_id
        )
        labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

        predictions_gathered = accelerator.gather(generated_tokens)
        labels_gathered = accelerator.gather(labels)

        decoded_preds, decoded_labels = postprocess(predictions_gathered, labels_gathered)
        metric.add_batch(predictions=decoded_preds, references=decoded_labels)

    losses = torch.cat(losses)
    losses = losses[: len(eval_dataloader)]

    results = metric.compute()
    belu_score = round(results['score'],2)
    training_loss = round(avg_train_loss,2)
    validation_loss = round(torch.mean(losses),2)
    print(f"epoch {epoch}")
    print(f"BLEU score: {belu_score}")
    print(f"Training Loss : {training_loss}")
    print(f"Validation Loss : {validation_loss}")
    # Log perplexity to wandb
    wandb.log({"epoch":epoch,"BLEU Score": belu_score,
               "Training Loss":training_loss,"Validation Loss":validation_loss})

    # Update model's config with BELU
    model.config.belu_score = belu_score
    model.config.epochs = epoch
    model.config.training_loss = training_loss
    nodel.config.validation_loss = validation_loss

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)

    # Update and save model's config
    unwrapped_model.config.save_pretrained(output_dir)

    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)
        repo.push_to_hub(
            commit_message=f"Training in progress epoch {epoch}", blocking=False
        )
# Close wandb run
wandb.finish()

  0%|          | 0/13500 [00:00<?, ?it/s]

KeyboardInterrupt: ignored

# Inference

In [37]:
from transformers import pipeline

# Replace this with your own checkpoint
infer_model_checkpoint = f"pritam3355/{model_name}"
translator = pipeline("translation", model=model_checkpoint)


/usr/local/lib/python3.10/dist-packages/transformers/pipelines/__init__.py:972: UserWarning: "translation" task was used, instead of "translation_XX_to_YY", defaulting to "translation_en_to_de"
  warnings.warn(


[{'translation_text': 'Standardmäßig auf erweiterte Threads'}]

In [38]:
translator("Default to expanded threads")

[{'translation_text': 'Standardmäßig auf erweiterte Threads'}]